In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
import mlflow
import numpy as np
from azureml.core import Workspace

In [2]:
# connect to your workspace
ws = Workspace.from_config()

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/ETSISI-OGVD/practicaogvd23-24-team-b/main/data/apple_quality.csv', sep=',')

df.dropna(inplace=True)

df.drop('A_id', axis=1, inplace=True)

df = df[df['Acidity'] != 'Created_by_Nidula_Elgiriyewithana']
df['Acidity'] = df['Acidity'].astype(float)

le = LabelEncoder()
df['Quality'] = le.fit_transform(df['Quality'])

In [4]:
X = df.drop('Quality', axis=1)
y = df['Quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# create experiment and start logging to a new run in the experiment
experiment_name = "azure-ml-apple-quality"

# set up MLflow to track the metrics
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)
mlflow.autolog()

# set up the Logistic regression model
mlp_clf = MLPClassifier(hidden_layer_sizes=(100, 50), learning_rate_init=0.075)

# train the model
with mlflow.start_run() as run:
    scores = cross_val_score(mlp_clf, X, y, cv=5) 

print("MLP Classifier cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())

# register the model
model_uri = "runs:/{}/model".format(run.info.run_id)
model = mlflow.register_model(model_uri, "sklearn_apple_model")

2024/03/15 15:06:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/03/15 15:06:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/03/15 15:06:43 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: UserError: Resource Conflict: ArtifactId ExperimentRun/dcid.432b8fd0-4961-4ed2-9e06-56c2f51ece24/training_confusion_matrix.png already exists.
UserError: Resource Conflict: ArtifactId ExperimentRun/dcid.432b8fd0-4961-4ed2-9e06-56c2f51ece24/training_precision_recall_curve.png already exists.
UserError: Resource Conflict: ArtifactId ExperimentRun/dcid.432b8fd0-4961-4ed2-9e06-56c2f51ece24/training_roc_curve.png already exists.
2024/03/15 15:06:46 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologg

MLP Classifier cross-validation scores: [0.9325 0.9325 0.9375 0.9175 0.94  ]
Mean accuracy: 0.932
